In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize 

In [2]:
df = pd.read_csv("/Users/teja/Downloads/netflix_titles.csv")
df.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [5]:
#create a new feature that we will use to calculate the similarity between movies, it's a combination of some other features
df["overall_infos"]= df["type"] + " "+df["title"]+" "+df["description"]

In [10]:
#adding new column because there are mistakes in show_id column
df.insert(1, "id", list(range(1, len(df)+1)), True) 

In [12]:
df.head()

,show_id,id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,overall_infos
0,s1,1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,TV Show 3% In a future where the elite inhabit...
1,s2,2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,Movie 7:19 After a devastating earthquake hits...
2,s3,3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",Movie 23:59 When an army recruit is found dead...
3,s4,4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...","Movie 9 In a postapocalyptic world, rag-doll r..."
4,s5,5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,Movie 21 A brilliant group of students become ...


In [23]:
#
df_processed = df[['id','overall_infos']]
df_processed.head(1)

,id,overall_infos
0,1,TV Show 3% In a future where the elite inhabit...


In [21]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [22]:
#processing the overall info column
def text_preprocessing(column):
    #make all words with lower letters
    column = column.str.lower()
    #getting rid of any punctution
    column = column.str.replace('http\S+|www.\S+|@|%|:|,|', '', case=False)
    #spliting each sentence to words to apply previous funtions on them 
    word_tokens = column.str.split()
    keywords = word_tokens.apply(lambda x: [item for item in x if item not in stop])
    #assemble words of each sentence again and assign them in new column
    for i in range(len(keywords)):
        keywords[i] = " ".join(keywords[i])
        column = keywords

    return column

In [24]:
df_processed['processed_infos'] = text_preprocessing(df_processed['overall_infos'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_processed.head(1)

,id,overall_infos,processed_infos
0,1,TV Show 3% In a future where the elite inhabit...,tv show 3 future elite inhabit island paradise...


In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

CV = CountVectorizer()
converted_metrix = CV.fit_transform(df_processed['processed_infos'])

In [30]:
cosine_similarity = cosine_similarity(converted_metrix)

In [31]:
cosine_similarity

array([[1.        , 0.        , 0.05892557, ..., 0.        , 0.10660036,
        0.        ],
       [0.        , 1.        , 0.0571662 , ..., 0.17149859, 0.        ,
        0.05057217],
       [0.05892557, 0.0571662 , 1.        , ..., 0.05555556, 0.        ,
        0.04914732],
       ...,
       [0.        , 0.17149859, 0.05555556, ..., 1.        , 0.        ,
        0.09829464],
       [0.10660036, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.05057217, 0.04914732, ..., 0.09829464, 0.        ,
        1.        ]])

In [46]:
#recommendation based on title, type, description
def get_recommendations(title):    
    if title is not None:
        recom = df[df['overall_infos'].str.contains(title)]
    return recom

In [47]:
title = input("Enter title name: ")
get_recommendations(title)

Enter title name: LEGO


,show_id,id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,overall_infos
3578,s3579,3579,TV Show,LEGO Bionicle: The Journey to One,NaN,"Nolan Balzer, Paolo Bryant, Jacqui Fox, Quinn ...",United States,"July 29, 2016",2016,TV-Y7,2 Seasons,Kids' TV,Six legendary heroes find themselves on an epi...,TV Show LEGO Bionicle: The Journey to One Six ...
3580,s3581,3581,TV Show,LEGO Elves: Secrets of Elvendale,NaN,"Ashleigh Ball, Erin Mathews, Rebecca Husain, K...",United States,"September 1, 2017",2017,TV-Y7,1 Season,Kids' TV,"Armed with a powerful amulet, a teenage guardi...",TV Show LEGO Elves: Secrets of Elvendale Armed...
3583,s3584,3584,TV Show,LEGO Friends: The Power of Friendship,NaN,"Erica Mendez, Marianne Miller, Rachelle Heger,...",Denmark,"June 10, 2016",2016,TV-Y7,2 Seasons,Kids' TV,Five best friends face adventures side by side...,TV Show LEGO Friends: The Power of Friendship ...
3584,s3585,3585,Movie,LEGO House - Home of the Brick,"Anders Falck, Stinus Morell Vithner",Bjarke Ingels,NaN,"June 15, 2018",2018,TV-G,48 min,"Documentaries, International Movies","In Billund, Denmark, a nearly 130,000-square-f...",Movie LEGO House - Home of the Brick In Billun...
3585,s3586,3586,TV Show,LEGO Jurassic World: Legend of Isla Nublar,NaN,NaN,United States,"November 1, 2020",2019,TV-Y,1 Season,Kids' TV,"At Jurassic Park, a velociraptor handler and a...",TV Show LEGO Jurassic World: Legend of Isla Nu...
3586,s3587,3587,TV Show,LEGO Jurassic World: Secret Exhibit,NaN,"Britt McKillip, Ian Hanlin, Dhirendra, Alex Za...",United States,"November 1, 2020",2018,TV-Y,1 Season,Kids' TV,A can-do problem solver teams up with a rugged...,TV Show LEGO Jurassic World: Secret Exhibit A ...
3587,s3588,3588,Movie,LEGO Jurassic World: The Indominus Escape,Michael D. Black,"A.J. LoCascio, Sendhil Ramamurthy, Fred Tatasc...",United States,"October 4, 2016",2016,TV-Y7,25 min,"Children & Family Movies, Comedies",The scientists of Jurassic World create a capt...,Movie LEGO Jurassic World: The Indominus Escap...
3588,s3589,3589,Movie,LEGO Marvel Super Heroes: Avengers Reassembled!,Rob Silvestri,"Laura Bailey, Troy Baker, Eric Bauza, Ben Disk...",United States,"August 6, 2017",2015,TV-Y7,22 min,Children & Family Movies,When Ultron seizes control of Iron Man's armor...,Movie LEGO Marvel Super Heroes: Avengers Reass...
3589,s3590,3590,Movie,LEGO Marvel Super Heroes: Black Panther,Michael D. Black,"James Mathis III, Keston John, Yvette Nicole B...",United States,"February 1, 2019",2018,TV-Y7,22 min,Children & Family Movies,When Thanos joins forces with villains Killmon...,Movie LEGO Marvel Super Heroes: Black Panther ...
3590,s3591,3591,Movie,LEGO Marvel Super Heroes: Guardians of the Galaxy,NaN,NaN,United States,"February 1, 2019",2017,TV-Y,22 min,Children & Family Movies,The Guardians are on a mission to deliver the ...,Movie LEGO Marvel Super Heroes: Guardians of t...


In [49]:
#this how we will get the id of the movie so we can check similarity between it and other movies
title = 'Teenage Mutant Ninja Turtles'

movie_id = df[df['title'] == title]['id'].values[0]

In [55]:
#simil
score = list(enumerate(cosine_similarity[movie_id]))

In [57]:
#now sort the similar movies in descending order
sorted_score = sorted(score, key=lambda x:x[1], reverse= True)
#we will ignore the first score because it will give us a 100% score because it's the same movie 
sorted_score = sorted_score[1:]

In [58]:
#now showing the top 5 movies similar to TMNT accroding to this algorithm
i = 0
for item in sorted_score:
    movie_title = df[df['id'] == item[0]]['title'].values[0]
    print(i+1,movie_title)
    i = i+1
    if i > 4:
        break

1 Teenage Mutant Ninja Turtles II: The Secret of the Ooze
2 Teenage Cocktail
3 Ninja Hattori
4 Power Rangers Ninja Steel
5 Power Rangers Mystic Force
